In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim

In [2]:
class Attention(nn.Module):
  def __init__(self, dim1, dim2):
    super(Attention, self).__init__()
    self.Queries = nn.Linear(dim2, dim2)
    self.Keys = nn.Linear(dim2, dim2)
    self.Values = nn.Linear(dim2, dim2)
    self.size = dim1
  def forward(self, Query, Key, Value):
    Query = self.Queries(Query)
    Key = self.Keys(Key)
    Value = self.Values(Value)
    x = torch.mm(Query, torch.t(Key))
    x = torch.softmax(x/(self.size**(1/2)), dim=1)
    attention = torch.mm(x, Value)
    return attention

In [3]:
class Encoder(nn.Module):
  def __init__(self, dim1, dim2, for_dim, no_heads):
    super(Encoder, self).__init__()
    self.dim1 = dim1
    self.dim2 = dim2
    self.Attention = Attention(dim1, dim2)
    self.no_heads = no_heads
    self.norm = nn.LayerNorm(dim2)
    self.feedforward = self.feed_forward(dim2, for_dim)
  def feed_forward(self, dimension1, dimension2):
    return nn.Sequential(
        nn.Linear(dimension1, dimension2),
        nn.ReLU(),
        nn.Linear(dimension2, dimension1)
    )
  def forward(self, Data):
    x = torch.tensor([])
    for i in range(self.no_heads):
      x = torch.cat((x, self.Attention(Data, Data, Data)), 0)
    x1 = Data
    for i in range(self.no_heads - 1):
      x1 = torch.cat((x1, Data), 0)
    x = self.norm(x + x1)
    x2 = torch.reshape(x, (self.no_heads, self.dim1, self.dim2))
    x = torch.mean(x2, 0)
    x1 = self.feedforward(x)
    return self.norm(x + x1)

In [4]:
class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.Enc = Encoder(240, 16, 32, 2)
    self.Flatten = nn.Flatten(0, -1)
    self.Layer1 = nn.Linear(3840, 256)
    self.Layer2 = nn.Linear(256, 128)
    self.Layer3 = nn.Linear(128, 64)
    self.Layer4 = nn.Linear(64, 16)
    self.Layer5 = nn.Linear(16, 3)
    self.Drop = nn.Dropout(p=0.25)
  def forward(self, x):
    x = self.Enc(x)
    x = self.Enc(x)
    x = self.Enc(x)
    x = self.Enc(x)
    x = self.Flatten(x)
    x = self.Drop(F.relu(self.Layer1(x)))
    x = self.Drop(F.relu(self.Layer2(x)))
    x = F.relu(self.Layer3(x))
    x = F.relu(self.Layer4(x))
    x = torch.special.expit(self.Layer5(x))
    return x

In [5]:
import scipy.io
daata = scipy.io.loadmat('/content/drive/MyDrive/Data/new_data.mat')
data1 = daata['data']

In [6]:
import scipy.io
daata = scipy.io.loadmat('/content/drive/MyDrive/Data/labels.mat')
label = daata['labels']

In [7]:
data = np.zeros((11281,240,16))
for i in range(0,11281):
  data[i,:,0] = data1[i,:,21]
  data[i,:,1] = data1[i,:,23]
  data[i,:,2] = data1[i,:,8]
  data[i,:,3] = data1[i,:,12]
  data[i,:,4] = data1[i,:,46]
  data[i,:,5] = data1[i,:,54]
  data[i,:,6] = data1[i,:,60]
  data[i,:,7] = data1[i,:,62]
  data[i,:,8] = data1[i,:,29]
  data[i,:,9] = data1[i,:,37]
  data[i,:,10] = data1[i,:,31]
  data[i,:,11] = data1[i,:,35]
  data[i,:,12] = data1[i,:,40]
  data[i,:,13] = data1[i,:,41]
  data[i,:,14] = data1[i,:,48]
  data[i,:,15] = data1[i,:,52]
data = (data+1)/2

In [ ]:
Data = []
Label = []
for i in range(11281):
  print(i)
  Data.append(torch.tensor(data[i]).float())
  Label.append(torch.tensor(label[i]).float())

In [9]:
Data = torch.stack(Data)
Label = torch.stack(Label)

In [ ]:
net = Model()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)
criterion = nn.MSELoss()
loss_ = []
for i in range(10):
  sol = []
  for j in range(1281):
    print(j)
    sol.append(net(Data[j]))
  sol = torch.stack(sol)
  loss = criterion(sol, Label[0:1281])
  net.zero_grad()
  loss.backward()
  optimizer.step()
  loss_.append(int(loss.item()))
  print('Loss in Iteration No.' + str(i) + ' is ' + str(loss.item()))